In [ ]:
from selenium import webdriver
from selenium.webdriver.support.ui import Select # Selectタグが扱えるエレメントに変化させる為の関数を呼び出す
from selenium.webdriver.common.action_chains import ActionChains
import pandas as pd
import time
import random
import getpass # 実行ユーザーを取得するために使用
import requests
import json
import datetime
import schedule

In [ ]:
def exhibition():
    print(datetime.datetime.now())
    print("exhibition")
    
    driver = webdriver.Chrome()#指定の位置にwebdriverのpathを書いてください　例）/Users/ユーザー名/chromedriver
    driver.get('https://www.mercari.com/jp/login/')
    f=open('mercari.txt','a')
    f.write("mercariにアクセスしました\n")
    f.close()
    time.sleep(1)

    #メアドとパスワードを入力
    driver.find_element_by_name('email').send_keys('mr.msk@icloud.com')#メールアドレスを入力して下さい
    time.sleep(1)
    driver.find_element_by_name('password').send_keys('070708Masaki')#パスワードを入力してください
    time.sleep(1)

    #recaptureのtextareaとログインボタンをウインドウ内に表示するために最下部にスクロール
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    #recapture対策
    driver.execute_script('var element=document.getElementById("g-recaptcha-response"); element.style.display="";')
    service_key = '' # 2captchaのservice key を入力してください
    google_site_key = '6LfZGCYTAAAAAHoK-s7Lg5OdYg5Fq4susvALmZoc' # reCAPTCHAのdata-sitekey
    pageurl = 'https://www.mercari.com/jp/login/' #recaptureのあるサイトurl
    url = "http://2captcha.com/in.php?key=" + service_key + "&method=userrecaptcha&googlekey=" + google_site_key + "&pageurl=" + pageurl 
    resp = requests.get(url) 
    if resp.text[0:2] != 'OK': 
        quit('Service error. Error code:' + resp.text) 
    captcha_id = resp.text[3:]
    fetch_url = "http://2captcha.com/res.php?key="+ service_key + "&action=get&id=" + captcha_id
    for i in range(1, 10):
        time.sleep(5) # wait 5 sec.
        resp = requests.get(fetch_url)
        if resp.text[0:2] == 'OK':
            break
    print('Google response token: ', resp.text[3:])
    driver.find_element_by_id('g-recaptcha-response').send_keys(resp.text[3:])# textareaへトークン入力
    time.sleep(10)#トークン入力に時間がかかるから長めに取っておく
    #ログインボタンを押す（textareaが邪魔でボタンを押せないのでクリック位置を下に1ずらしている
    target=driver.find_element_by_xpath('/html/body/div[1]/main/div/form/div/button')
    actions=ActionChains(driver)
    actions.move_to_element(target)
    actions.move_by_offset(0,1)#targetから下に1マウスカーソル移動
    print(target.location)
    actions.click()
    actions.perform()#実行
    time.sleep(random.randint(2,5))
    f=open('mercari.txt','a')
    f.write("mercariにログインしました\n")
    f.close()

    #出品する商品情報の取得
    csv_input = pd.read_csv(filepath_or_buffer="book.csv", encoding="shift_jis", sep=",")
    print(csv_input.values)
    #出品ボタン
    driver.find_element_by_class_name("icon-camera").click()

    time.sleep(1)

    #商品画像
    driver.find_element_by_xpath( "//input[@type='file']" ).send_keys("C:/Users/care-lab/Desktop/mercari/photo1.jpg")

    #商品名
    name=driver.find_element_by_xpath( "//*[@id='sell-container']/div/div/form/div[2]/div[1]/div/input" )
    name.send_keys(csv_input.values[0][0])

    #商品の説明
    description=driver.find_element_by_xpath( "//*[@id='sell-container']/div/div/form/div[2]/div[2]/textarea" )
    description.send_keys(csv_input.values[0][1])

    #カテゴリ
    # エレメントの取得
    category1= driver.find_element_by_xpath("//*[@id='sell-container']/div/div/form/div[3]/div/div[1]/div/div/select")
    # 取得したエレメントをSelectタグに対応したエレメントに変化させてvalueを指定する
    Select(category1).select_by_value(str(csv_input.values[0][2]))
    category2 = driver.find_element_by_xpath("//*[@id='sell-container']/div/div/form/div[3]/div/div[1]/div/div[2]/select")
    Select(category2).select_by_value(str(csv_input.values[0][3]))
    category3 = driver.find_element_by_xpath("//*[@id='sell-container']/div/div/form/div[3]/div/div[1]/div/div[3]/select")
    Select(category3).select_by_value(str(csv_input.values[0][4]))

    #商品の状態
    status = driver.find_element_by_xpath("//*[@id='sell-container']/div/div/form/div[3]/div/div[2]/div/select")
    Select(status).select_by_value(str(csv_input.values[0][5]))

    #配送料の負担
    ship = driver.find_element_by_xpath("//*[@id='sell-container']/div/div/form/div[4]/div/div[1]/div/select")
    Select(ship).select_by_value(str(csv_input.values[0][6]))

    #配送の方法
    method = driver.find_element_by_xpath("//*[@id='sell-container']/div/div/form/div[4]/div/div[2]/div/select")
    Select(method).select_by_value(str(csv_input.values[0][7]))

    #発送元の地域
    region = driver.find_element_by_xpath("//*[@id='sell-container']/div/div/form/div[4]/div/div[3]/div/select")
    Select(region).select_by_value(str(csv_input.values[0][8]))

    #発送までの日数
    day = driver.find_element_by_xpath("//*[@id='sell-container']/div/div/form/div[4]/div/div[4]/div/select")
    Select(day).select_by_value(str(csv_input.values[0][9]))

    #販売価格
    value=driver.find_element_by_xpath( "//*[@id='sell-container']/div/div/form/div[5]/div/ul/li[1]/div/div/div/input" )
    value.send_keys(csv_input.values[0][10])

    time.sleep(1)

    #driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    #出品ボタン
    driver.find_element_by_xpath("//*[@id='sell-container']/div/div/form/div[7]/button").click()
    
    time.sleep(5)
        
    driver.close()
    

In [ ]:
def cancel():
    print(datetime.datetime.now())
    print("cancel")
    
    driver = webdriver.Chrome()#指定の位置にwebdriverのpathを書いてください　例）/Users/ユーザー名/chromedriver
    driver.get('https://www.mercari.com/jp/login/')
    f=open('mercari.txt','a')
    f.write("mercariにアクセスしました\n")
    f.close()
    time.sleep(1)

    #メアドとパスワードを入力
    driver.find_element_by_name('email').send_keys('')#メールアドレスを入力して下さい
    time.sleep(1)
    driver.find_element_by_name('password').send_keys('')#パスワードを入力してください
    time.sleep(1)

    #recaptureのtextareaとログインボタンをウインドウ内に表示するために最下部にスクロール
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    #recapture対策
    driver.execute_script('var element=document.getElementById("g-recaptcha-response"); element.style.display="";')
    service_key = '' # 2captchaのservice key 
    google_site_key = '6LfZGCYTAAAAAHoK-s7Lg5OdYg5Fq4susvALmZoc' # reCAPTCHAのdata-sitekey
    pageurl = 'https://www.mercari.com/jp/login/' #recaptureのあるサイトurl
    url = "http://2captcha.com/in.php?key=" + service_key + "&method=userrecaptcha&googlekey=" + google_site_key + "&pageurl=" + pageurl 
    resp = requests.get(url) 
    if resp.text[0:2] != 'OK': 
        quit('Service error. Error code:' + resp.text) 
    captcha_id = resp.text[3:]
    fetch_url = "http://2captcha.com/res.php?key="+ service_key + "&action=get&id=" + captcha_id
    for i in range(1, 10):
        time.sleep(5) # wait 5 sec.
        resp = requests.get(fetch_url)
        if resp.text[0:2] == 'OK':
            break
    print('Google response token: ', resp.text[3:])
    driver.find_element_by_id('g-recaptcha-response').send_keys(resp.text[3:])# textareaへトークン入力
    time.sleep(10)#トークン入力に時間がかかるから長めに取っておく
    
    #ログインボタンを押す（textareaが邪魔でボタンを押せないのでクリック位置を下に1ずらしている
    target=driver.find_element_by_xpath('/html/body/div[1]/main/div/form/div/button')
    actions=ActionChains(driver)
    actions.move_to_element(target)
    actions.move_by_offset(0,1)#targetから下に1マウスカーソル移動
    print(target.location)
    actions.click()
    actions.perform()#実行
    time.sleep(random.randint(2,5))
    f=open('mercari.txt','a')
    f.write("mercariにログインしました\n")
    f.close()

    #出品一覧ページ
    mypage=driver.find_element_by_class_name("mypage-nav-list").find_elements_by_tag_name('a')[1].get_attribute("href")
    driver.get(mypage)
    #出品個別ページ
    driver.find_element_by_class_name('mypage-tab-container').find_elements_by_class_name('mypage-item-link')[0].click()
    #削除ボタン
    driver.find_element_by_css_selector('body > div > main > div.l-content > div.listing-item-change-box > button').click()
    #削除確認ボタン
    driver.find_element_by_css_selector('body > div > main > div.l-content > aside > div > div.clearfix > form > button').click()
    
    time.sleep(5)

    driver.close()

In [ ]:
#schedule.every().day.at("23:47").do(exhibition)
#schedule.every().day.at("23:49").do(cancel)  

while True:
    #exhibition()
    schedule.run_pending()
    time.sleep(6)
